In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/urdu-news-data/description.csv
/kaggle/input/urdu-news-data/headline.csv
/kaggle/input/urdu-stop-word/stopword.txt


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.layers import LSTM
import numpy as np
import pandas as pd
import re

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Define maximum sequence length
MAX_SEQUENCE_LENGTH = 1000


In [4]:
# Load the CSV file
data = pd.read_csv('/kaggle/input/urdu-news-data/description.csv',engine='python',encoding='utf-8')

data = data.dropna()

In [5]:
stop_word_list = []
with open("/kaggle/input/urdu-stop-word/stopword.txt") as fin:
    for i in fin:
        stop_word_list.append(i.strip())
fin.close()

pattern = r'[a-zA-Z0-9!?:;"(,)]'
data['Description'] = data['Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_word_list]))
data['Description'] = data['Description'].apply(lambda x: re.sub(pattern,'',x))

In [6]:
descriptions = data['Description'].tolist()
categories = data['Category'].tolist()

In [7]:
# Tokenize text
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(descriptions)
sequences = tokenizer.texts_to_sequences(descriptions)

# Pad sequences
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [8]:
# Encode categories
label_encoder = LabelEncoder()
integer_encoded_labels = label_encoder.fit_transform(categories)
num_classes = len(label_encoder.classes_)


In [9]:
# Split data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(padded_sequences, integer_encoded_labels, test_size=0.2)

In [10]:
model = tf.keras.Sequential([
    Embedding(5000, 128, input_length=MAX_SEQUENCE_LENGTH),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(num_classes, activation="softmax"),
])
# Compile model
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)


In [11]:
# Train the model
model.fit(
    train_data,
    train_labels,
    epochs=10,
    validation_data=(test_data, test_labels),
)

Epoch 1/10
4431/4431 [==============================] - 413s 92ms/step - loss: 1.2405 - accuracy: 0.5317 - val_loss: 1.1331 - val_accuracy: 0.5802
Epoch 2/10
4431/4431 [==============================] - 379s 86ms/step - loss: 1.0735 - accuracy: 0.5990 - val_loss: 1.0878 - val_accuracy: 0.5882
Epoch 3/10
4431/4431 [==============================] - 379s 85ms/step - loss: 1.0125 - accuracy: 0.6130 - val_loss: 1.0811 - val_accuracy: 0.5877
Epoch 4/10
4431/4431 [==============================] - 378s 85ms/step - loss: 0.9638 - accuracy: 0.6244 - val_loss: 1.0887 - val_accuracy: 0.5847
Epoch 5/10
4431/4431 [==============================] - 378s 85ms/step - loss: 0.9210 - accuracy: 0.6338 - val_loss: 1.0954 - val_accuracy: 0.5814
Epoch 6/10
4431/4431 [==============================] - 378s 85ms/step - loss: 0.8769 - accuracy: 0.6441 - val_loss: 1.1242 - val_accuracy: 0.5771
Epoch 7/10
4431/4431 [==============================] - 378s 85ms/step - loss: 0.8328 - accuracy: 0.6566 - val_loss: 1

In [12]:
# Predict on test data
predictions = model.predict(test_data)

# Generate classification report
print("RNN Report")
y_pred = np.argmax(predictions, axis=1)
print(classification_report(test_labels, y_pred))

1108/1108 [==============================] - 39s 34ms/step
RNN Report
              precision    recall  f1-score   support

           0       0.04      0.01      0.01       176
           1       0.68      0.73      0.70      6242
           2       0.44      0.44      0.44      6398
           3       0.53      0.58      0.55      5101
           4       0.44      0.35      0.39      6437
           5       0.73      0.81      0.77      5023
           6       0.48      0.48      0.48      6072

    accuracy                           0.55     35449
   macro avg       0.48      0.48      0.48     35449
weighted avg       0.54      0.55      0.54     35449

